# Practice notebook for univariate analysis using NHANES data

# Example Solutions

**This notebook contains one or or more code examples for the practice questions.**

**Your code might be different. But the output should be the same.**

**Do you have a alternate solution to a question? Share it with your peers on the course forum!**

This notebook will give you the opportunity to perform some univariate analyses on your own using the NHANES data.  These analyses are similar to what was done in the week 2 NHANES case study notebook.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [ ]:
# import the usual libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
# set the display.max_columns option to None so that all columns will display
pd.set_option('display.max_columns', None)

# read the nhanes csv file
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# browse the variables
# the variable code book is available at https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm
da.columns

In [ ]:
# browse the data
da.head(10)

## Question 1

Relabel the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to have brief but informative character labels.  Then construct a frequency table of these values for all people, then for women only and for men only.  Then construct these three frequency tables using only people whose age is between 30 and 40.

https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm

https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

https://pandas.pydata.org/docs/reference/api/pandas.cut.html

https://pandas.pydata.org/docs/reference/api/pandas.interval_range.html

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# relabel DMDMARTL
rmar = {1: "Married", 2: "Widowed", 3: "Divorced", 4: "Separated", 5: "NeverMarried", 6: "Cohabitating", 77: "Refused", 99: "Unknown"}
da_mod["DMDMARTLx"] = da_mod["DMDMARTL"].replace(rmar)

# relabel RIAGENDR
rgen = {1: "Male", 2: "Female"}
da_mod["RIAGENDRx"] = da_mod["RIAGENDR"].replace(rgen)

# drop the variables that we don't need
da_mod = da_mod[["RIAGENDRx", "RIDAGEYR", "DMDMARTLx"]]

In [ ]:
# frequency table for all subjects
# uncomment print(x / x.sum()) for proportions instead of counts
print("All Subjects")
x = da_mod["DMDMARTLx"].value_counts()
#print(x / x.sum())
print(x)

# frequency table by gender
for ky,db in da_mod.groupby("RIAGENDRx"):
    print("\nRIAGENDRx =", ky)
    x = db["DMDMARTLx"].value_counts()
    #print(x / x.sum())
    print(x)

# create new dataframe restricted to subjects between 30 and 40 (inclusive)
da3040 = da_mod.query('RIDAGEYR >= 30 & RIDAGEYR <= 40').copy()

# frequency table for all subjects
print("\nAll Subjects 30 <= RIDAGEYR <= 40")
x = da3040["DMDMARTLx"].value_counts()
#print(x / x.sum())
print(x)

# frequency table by RIAGENDRx
for ky,db in da3040.groupby("RIAGENDRx"):
    print("\nRIAGENDRx =", ky, "30 <= RIDAGEYR <= 40")
    x = db["DMDMARTLx"].value_counts()
    #print(x / x.sum())
    print(x)

# another way to generate the age-restricted frequency table...

# create a closed interval range from 30 to 40
#binrange = pd.interval_range(start=30, end=40, periods=1, closed='both')

# create the 30-40 'bin' variable
#da_mod["agegrp"] = pd.cut(da_mod.RIDAGEYR, binrange)

# what does agegroup look like?
#print(da_mod.agegrp.value_counts(dropna=False))

# frequency table for all subjects
#x = da_mod.groupby(["agegrp"])["DMDMARTLx"].value_counts()
#print("\nAll Subjects 30 <= RIDAGEYR <= 40")
#print(x)

# frequency table by RIAGENDRx
#x = da_mod.groupby(["agegrp", "RIAGENDRx"])["DMDMARTLx"].value_counts()
#print("\nMales and Females 30 <= RIDAGEYR <= 40")
#print(x)

__Q1a.__ What are the differences (if any) that you observe in the distribution of marital status between females and males?



__Q1b.__ What are the differences (if any) that you observe in the distribution of marital status between females overall and females aged 30 to 40?

__Q1c.__ Repeat part Q1b for males.

## Question 2

Restricting to the female population, stratify the subjects into age bands no wider than ten years, and construct the distribution of marital status within each age band.  Within each age band, present the distribution in terms of proportions that must sum to 1.

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# relabel DMDMARTL
rmar = {1: "Married", 2: "Widowed", 3: "Divorced", 4: "Separated", 5: "NeverMarried", 6: "Cohabitating", 77: "Refused", 99: "Unknown"}
da_mod["DMDMARTLx"] = da_mod["DMDMARTL"].replace(rmar)

# drop the variables that we don't need
da_mod = da_mod[["RIAGENDR", "RIDAGEYR", "DMDMARTLx"]]

In [ ]:
# one possible solution uses the math library
import math

# restrict the dataset to females only
da_mod = da_mod.query('RIAGENDR == 2').copy()

# calculate the minimum and maximum ages
minage = min(da_mod.RIDAGEYR)
maxage = max(da_mod.RIDAGEYR)

# one way to create the list of bins so that the first bin is closed on the left
bins = [minage] + list(range(int(math.ceil(minage/10)) * 10, int(math.ceil(maxage/10)) * 10, 10)) + [maxage]

# another way to create the list of bins without using the math library
# bins = [18, 20, 30, 40, 50, 60, 70, 80]
#print(bins)

# create labels for the bins
labels = [f"({bins[i]}, {bins[i+1]}]" if(i) else f"[{bins[i]}, {bins[i+1]}]" for i in range(len(bins)-1)]

# create the agegrp "bins" variable
da_mod["agegrp"] = pd.cut(da_mod.RIDAGEYR, bins, labels=labels, include_lowest = True)

# what does agegroup look like?
#print(da_mod.agegrp.value_counts(dropna=False))

# what does DMDMARTLx look like? Any rare or missing values?
#print(da_mod.DMDMARTLx.value_counts(dropna=False))

# distribution of marital status
dx = da_mod.loc[~da_mod.DMDMARTLx.isin(["Refused", np.nan]), :]  # drop rare/missing values
dx = dx.groupby(["agegrp"])["DMDMARTLx"]
dx = dx.value_counts()
dx = dx.unstack() # Restructure the results from 'long' to 'wide'
dx = dx.apply(lambda x: x/x.sum(), axis=1) # Normalize within each stratum to get proportions
print("Females")
print(dx.to_string(float_format="%.3f"))  # Limit display to 3 decimal places

# alternative that replaces NaN with 0
#print(dx.fillna(0).to_string(float_format="%.3f"))

__Q2a.__ What trends (if any) do you see in this series of marginal distributions?

__Q2b.__ Repeat the analysis for males.

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# relabel DMDMARTL
rmar = {1: "Married", 2: "Widowed", 3: "Divorced", 4: "Separated", 5: "NeverMarried", 6: "Cohabitating", 77: "Refused", 99: "Unknown"}
da_mod["DMDMARTLx"] = da_mod["DMDMARTL"].replace(rmar)

# drop the variables that we don't need
da_mod = da_mod[["RIAGENDR", "RIDAGEYR", "DMDMARTLx"]]

In [ ]:
# one possible solution uses the math library
import math

# restrict the dataset to males only
da_mod = da_mod.query('RIAGENDR == 1').copy()

# calculate the minimum and maximum ages
minage = min(da_mod.RIDAGEYR)
maxage = max(da_mod.RIDAGEYR)

# one way to create the list of bins so that the first bin is closed on the left
bins = [minage] + list(range(int(math.ceil(minage/10)) * 10, int(math.ceil(maxage/10)) * 10, 10)) + [maxage]

# another (manual) way to create the list of bins without using the math library
# bins = [18, 20, 30, 40, 50, 60, 70, 80]
#print(bins)

# create labels for the bins
labels = [f"({bins[i]}, {bins[i+1]}]" if(i) else f"[{bins[i]}, {bins[i+1]}]" for i in range(len(bins)-1)]

# create the agegrp "bins" variable
da_mod["agegrp"] = pd.cut(da_mod.RIDAGEYR, bins, labels=labels, include_lowest = True)

# what does agegroup look like?
#print(da_mod.agegrp.value_counts(dropna=False))

# what does DMDMARTLx look like? Any rare or missing values?
#print(da_mod.DMDMARTLx.value_counts(dropna=False))

dx = da_mod.loc[~da_mod.DMDMARTLx.isin(["Refused", np.nan]), :]  # drop rare/missing values
dx = dx.groupby(["agegrp"])["DMDMARTLx"]
dx = dx.value_counts()
dx = dx.unstack() # Restructure the results from 'long' to 'wide'
dx = dx.apply(lambda x: x/x.sum(), axis=1) # Normalize within each stratum to get proportions
print("Males")
print(dx.to_string(float_format="%.3f"))  # Limit display to 3 decimal places

# alternative that replaces NaN with 0
#print(dx.fillna(0).to_string(float_format="%.3f"))

__Q2c.__ Comment on any notable differences that you see when comparing these results between females and for males.

## Question 3

Use histplot to generate a histogram of the distribution of heights using the [BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT) variable.

https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm

https://seaborn.pydata.org/generated/seaborn.histplot.html

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# relabel RIAGENDR
rgen = {1: "Male", 2: "Female"}
da_mod["RIAGENDRx"] = da_mod["RIAGENDR"].replace(rgen)

In [ ]:
# entire sample
sns.histplot(da_mod.BMXHT)

__Q3a.__ Use the bins argument to histplot to produce histograms with different numbers of bins.  Assess whether the default value for this argument gives a meaningful result. What happens as the number of bins grows excessively large or excessively small?

https://numpy.org/doc/stable/reference/generated/numpy.histogram_bin_edges.html#numpy.histogram_bin_edges

In [ ]:
# bins = 4
sns.histplot(da_mod.BMXHT, bins=4)

In [ ]:
# bins = 400
sns.histplot(da_mod.BMXHT, bins=400)

__Q3b.__ Make separate histograms for the heights of women and men, then make a side-by-side boxplot showing the heights of women and men.

https://seaborn.pydata.org/generated/seaborn.boxplot.html

In [ ]:
# histogram of female BMXHT
dafem = da_mod.query('RIAGENDR == 2')
sns.histplot(data=dafem.BMXHT)

In [ ]:
# histogram of male BMXHT
damal = da_mod.query('RIAGENDR == 1')
sns.histplot(data=damal.BMXHT)

In [ ]:
# boxplot of BMXHT by RIAGENDRx
sns.boxplot(data=da_mod, x="BMXHT", y="RIAGENDRx")

__Q3c.__ What features (if any) are not represented clearly in the boxplots, and what features (if any) are easier to see in the boxplots than in the histograms?

## Question 4

Make a boxplot showing the distribution of within-subject differences between the first and second systolic blood pressure measurents ([BPXSY1](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BPX_I.htm#BPXSY1) and [BPXSY2](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BPX_I.htm#BPXSY2)).

https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BPX_I.htm

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

In [ ]:
# boxplot of difference between BPXSY1 and BPXSY2
da_mod["BP Diff"] = da_mod.BPXSY1 - da.BPXSY2
sns.boxplot(data=da_mod[["BP Diff"]], orient="h")

__Q4a.__ What proportion of the subjects have a lower SBP on the second reading compared to the first?

In [ ]:
# create a boolean variable that is True if "BP Diff" is positive and False if "BP Diff" is negative
lower_second = (da_mod["BP Diff"].dropna() > 0)

# Booleans are either True or False but...
# internally Python stores True values as 1 and False values as 0...
# so taking the average of a Boolean variable will give us the proportion of 1 values (True values)
print(np.mean(lower_second))

# another way to measure the proportion of True and False values
#print(lower_second.value_counts(normalize=True))

__Q4b.__ Make side-by-side boxplots of the two systolic blood pressure variables.

In [ ]:
# boxplot to compare BPXSY1 and BPXSY2
sns.boxplot(data=da_mod.loc[:,["BPXSY1", "BPXSY2"]], orient='h')
plt.show()

__Q4c.__ Comment on the systolic blood pressure measurement variation (either BPXSY1 or BPXSY2) and the within-subject difference variation.

In [ ]:
# Numerical summaries might also be helpful in this instance
print(da_mod.BPXSY1.describe(), "\n")
print(da_mod.BPXSY2.describe(), "\n")
print(da_mod["BP Diff"].describe())

## Question 5

Construct a frequency table of household sizes for people within each educational attainment category (the relevant variable is [DMDEDUC2](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDEDUC2)).

Convert the frequencies to proportions.

https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm

https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

In [ ]:
# frequency (counts) table
pd.crosstab(da_mod.DMDEDUC2,da_mod.DMDHHSIZ)

In [ ]:
# normalized (by row) freqency table
pd.crosstab(da_mod.DMDEDUC2,da_mod.DMDHHSIZ, margins=True, normalize='index')

__Q5a.__ Comment on any major differences among the distributions.

__Q5b.__ Restrict the sample to people between 30 and 40 years of age.  Then calculate the median household size for women and men within each level of educational attainment.

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# relabel RIAGENDR
rgen = {1: "Male", 2: "Female"}
da_mod["RIAGENDRx"] = da_mod["RIAGENDR"].replace(rgen)

# relabel DMDEDUC2
redu = {1: "1) LT 9", 2: "2) 9 to 11", 3: "3) HS or GED", 4: "4) Some college or AA", 5: "5) College", 7: "7) Refused", 9: "9) Don't know"}
da_mod["DMDEDUC2x"] = da_mod.DMDEDUC2.replace(redu)

# drop the variables that we don't need
da_mod = da_mod[["RIAGENDRx", "RIDAGEYR", "DMDEDUC2x", "DMDHHSIZ"]]

In [ ]:
# create a closed interval range from 30 to 40
binrange = pd.interval_range(start=30, end=40, periods=1, closed='both')

# create the 30-40 'bin' variable
da_mod["agegrp"] = pd.cut(da_mod.RIDAGEYR, binrange)

# what does DMDEDUC2x look like?
#print(da_mod.DMDEDUC2x.value_counts())

# drop rare or missing values
da_mod = da_mod.loc[~da_mod.DMDEDUC2x.isin(["7) Refused", "9) Don't know"]), :]

# median by RIAGENDRx and DMDEDUC2x
dx = da_mod.groupby(["agegrp", "RIAGENDRx", "DMDEDUC2x"])["DMDHHSIZ"].median()
print(dx)

# another way to isolate subjects between 30 and 40 years old
#da3040 = da_mod.query('RIDAGEYR >= 30 & RIDAGEYR <= 40').copy()

# median by RIAGENDRx and DMDEDUC2x
#print(da3040.groupby(["RIAGENDRx", "DMDEDUC2x"])["DMDHHSIZ"].median())

## Question 6

The participants can be clustered into "masked variance units" (MVU) based on every combination of the variables [SDMVSTRA](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVSTRA) and [SDMVPSU](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVPSU).  Calculate the mean age ([RIDAGEYR](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIDAGEYR)), height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)), and BMI ([BMXBMI](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXBMI)) for each gender ([RIAGENDR](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIAGENDR)), within each MVU. Report the ratio between the largest and smallest mean (e.g. for height) for each gender across the MVUs.

Note: SDMVSTRA and SDMVPSU are not used this way in a real-world analysis. Their intended use within NHANES is for complex variation estimation.

https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html

In [ ]:
# if necessary reload the nhanes csv file to avoid possible data "contamination" from previous work
da = pd.read_csv("nhanes_2015_2016.csv")

In [ ]:
# make copy of the dataframe to protect the original
da_mod = da.copy()

# separate into two dataframes by RIAGENDRx
dam = da_mod.where(da_mod.RIAGENDR == 1)
daf = da_mod.where(da_mod.RIAGENDR == 2)

# drop the variables that we don't need
dam = dam[["SDMVSTRA", "SDMVPSU", "RIDAGEYR", "BMXHT", "BMXBMI"]]
daf = daf[["SDMVSTRA", "SDMVPSU", "RIDAGEYR", "BMXHT", "BMXBMI"]]

In [ ]:
# Analysis for Males

# calculate the mean for RIDAGEYR, BMXHT and BMXBMI for each MVU
da_mean = dam.groupby(['SDMVPSU', 'SDMVSTRA']).mean()

# calculate and print the BMXHT mean ratio
ratio_height_mean = da_mean['BMXHT'].max()/da_mean['BMXHT'].min()
print("Males")
print(round(ratio_height_mean, 3), " (Height)")

# calculate and print the RIDAGEYR mean ratio
ratio_age_mean = da_mean['RIDAGEYR'].max()/da_mean['RIDAGEYR'].min()
print(round(ratio_age_mean, 3), " (Age)")

# calculate and print the BMXBMI mean ratio
ratio_bmi_mean = da_mean['BMXBMI'].max()/da_mean['BMXBMI'].min()
print(round(ratio_bmi_mean, 3), " (BMI)")

# Analysis for Females

# calculate the mean for RIDAGEYR, BMXHT and BMXBMI across MVUs
da_mean = daf.groupby(['SDMVPSU', 'SDMVSTRA']).mean()

# calculate and print the BMXHT mean ratio
ratio_height_mean = da_mean['BMXHT'].max()/da_mean['BMXHT'].min()
print("\nFemales")
print(round(ratio_height_mean, 3), " (Height)")

# calculate and print the RIDAGEYR rmean atio
ratio_age_mean = da_mean['RIDAGEYR'].max()/da_mean['RIDAGEYR'].min()
print(round(ratio_age_mean, 3), " (Age)")

# calculate and print the BMXBMI mean ratio
ratio_bmi_mean = da_mean['BMXBMI'].max()/da_mean['BMXBMI'].min()
print(round(ratio_bmi_mean, 3), " (BMI)")

__Q6a.__ Comment on the extent to which mean age, height, and BMI vary across the MVUs for males and females.

__Q6b.__ Calculate the inter-quartile range (IQR) for age, height, and BMI for each gender and each MVU.  Report the ratio between the largest and smallest IQR across the MVUs.

https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.quantile.html


In [ ]:
# Males

# calculate the 2nd and 3rd quintiles for RIDAGEYR, BMXHT and BMXBMI for each MVU
da_q3 = dam.groupby(['SDMVPSU', 'SDMVSTRA']).quantile(.75, numeric_only=True)
da_q1 = dam.groupby(['SDMVPSU', 'SDMVSTRA']).quantile(.25, numeric_only=True)

# calculate the IQR for RIDAGEYR, BMXHT and BMXBMI
da_iqr = da_q3 - da_q1

# calculate and print the BMXHT iqr ratio
ratio_height_iqr = da_iqr['BMXHT'].max()/da_iqr['BMXHT'].min()
print("Males")
print(round(ratio_height_iqr, 3), " (Height)")

# calculate and print the RIDAGEYR iqr ratio
ratio_age_iqr = da_iqr['RIDAGEYR'].max()/da_iqr['RIDAGEYR'].min()
print(round(ratio_age_iqr, 3), " (Age)")

# calculate and print the BMXBMI iqr ratio
ratio_bmi_iqr = da_iqr['BMXBMI'].max()/da_iqr['BMXBMI'].min()
print(round(ratio_bmi_iqr, 3), " (BMI)")

# Females

# calculate the 2nd and 3rd quintiles for RIDAGEYR, BMXHT and BMXBMI across MVUs
da_q3 = daf.groupby(['SDMVPSU', 'SDMVSTRA']).quantile(.75, numeric_only=True)
da_q1 = daf.groupby(['SDMVPSU', 'SDMVSTRA']).quantile(.25, numeric_only=True)

# calculate the IQR for RIDAGEYR, BMXHT and BMXBMI
da_iqr = da_q3 - da_q1

# calculate and print the BMXHT iqr ratio
ratio_height_iqr = da_iqr['BMXHT'].max()/da_iqr['BMXHT'].min()
print("\nFemales")
print(round(ratio_height_iqr, 3), " (Height)")

# calculate and print the RIDAGEYR iqr ratio
ratio_age_iqr = da_iqr['RIDAGEYR'].max()/da_iqr['RIDAGEYR'].min()
print(round(ratio_age_iqr, 3), " (Age)")

# calculate and print the BMXBMI iqr ratio
ratio_bmi_iqr = da_iqr['BMXBMI'].max()/da_iqr['BMXBMI'].min()
print(round(ratio_bmi_iqr, 3), " (BMI)")


__Q6c.__ Comment on the extent to which the IQR for age, height, and BMI vary across the MVUs for males and females.